In [1]:
%load_ext autoreload
%autoreload 2

# Online Outcomes: Predicting Success in Virtual Learning

**<font size="4">Author: Jeffrey Hanif Watson</font>**

## Imports and Project Path

In [2]:
# importing packages
import os, sys, glob, re
import pandas as pd
import numpy as np
import sqlite3

# setting project path
gparent = os.path.join(os.pardir, os.pardir)
sys.path.append(gparent)

from src import functions as fn

import matplotlib.pyplot as plt
import seaborn as sns

## Creating SQL Database

**<font size="3">This cell can be run once to establish the database and then commented out.</font>**

In [3]:
# # creating and populating database
# file_name = 'anonymisedData.zip'
# database_name = 'outcomes.db'
# fn.db_create(file_name, database_name)

## Database ERD

![graph0](../../references/schema.png)

## EDA

### SQL Setup
Connecting to database and creating a cursor object.

In [4]:
# creating connection, and cursor
data_path = os.path.join(gparent,'data/processed','outcomes.db')
conn = sqlite3.connect(data_path)  
cur = conn.cursor()

### Checking table names

In [5]:
# checking the table names
q = """SELECT name FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1"""
fn.fetch(cur, q)

[('ASSESSMENTS',),
 ('COURSES',),
 ('STUDENTASSESSMENT',),
 ('STUDENTINFO',),
 ('STUDENTREGISTRATION',),
 ('STUDENTVLE',),
 ('VLE',)]

## Data Understanding

### Codes & Module Summaries

In [52]:
# month codes used in code_presentation
month_codes = {
    'A': 'January', 
    'B': 'Februrary',
    'C': 'March', 
    'D': 'April',
    'E': 'May',
    'F': 'June',
    'G': 'July',
    'H': 'August',
    'I': 'Septemeber',
    'J': 'October',
    'K': 'November',
    'L': 'December'
        }

In [38]:
module_codes = {
    'AAA': {'Domain': 'Social Sciences', 'Presentations': 2, 'Students': 748},
    'BBB': {'Domain': 'Social Sciences', 'Presentations': '4', 'Students': 7909},
    'CCC': {'Domain': 'STEM', 'Presentations': 2, 'Students': 4434},
    'DDD': {'Domain': 'STEM', 'Presentations': 4, 'Students': 6272},
    'EEE': {'Domain': 'STEM', 'Presentations': 3, 'Students': 2934},
    'FFF': {'Domain': 'STEM', 'Presentations': 4, 'Students': 7762},
    'GGG': {'Domain': 'Social Sciences', 'Presentations': 3, 'Students': 2534}
}

In [56]:
# checking work
print('Students')
print('---------')
for key in module_codes.keys():
    print(key, module_codes[key]['Students'])

Students
---------
AAA 748
BBB 7909
CCC 4434
DDD 6272
EEE 2934
FFF 7762
GGG 2534


### Creating Data Frames

### Student Info

In [6]:
# creating student_info dataframe and appending column names
q = "SELECT*FROM STUDENTINFO"
student_info = pd.DataFrame(fn.fetch(cur, q))
student_info.columns = [i[0] for i in cur.description]

In [7]:
# checking first five rows
student_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [17]:
student_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28785 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           28785 non-null  object
 1   code_presentation     28785 non-null  object
 2   id_student            28785 non-null  object
 3   gender                28785 non-null  object
 4   region                28785 non-null  object
 5   highest_education     28785 non-null  object
 6   imd_band              27814 non-null  object
 7   age_band              28785 non-null  object
 8   num_of_prev_attempts  28785 non-null  object
 9   studied_credits       28785 non-null  object
 10  disability            28785 non-null  object
 11  final_result          28785 non-null  object
dtypes: object(12)
memory usage: 2.9+ MB


In [8]:
student_info.shape

(32593, 12)

In [9]:
student_info.duplicated('id_student', keep=False).sum()

7346

In [10]:
student_info.drop_duplicates(subset='id_student', keep='first', inplace=True)

In [11]:
student_info.shape

(28785, 12)

Student VLE

In [12]:
# creating student_vle dataframe and appending column names
q = "SELECT*FROM STUDENTVLE"
student_vle = pd.DataFrame(fn.fetch(cur, q))
student_vle.columns = [i[0] for i in cur.description]

In [13]:
student_vle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [16]:
student_vle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         object
 3   id_site            object
 4   date               object
 5   sum_click          object
dtypes: object(6)
memory usage: 487.8+ MB


In [31]:
student_vle['sum_click'] = student_vle['sum_click'].astype('int')

In [32]:
student_sorted = student_vle.groupby('id_student')

In [35]:
clicks = student_sorted['sum_click'].sum()

In [57]:
clicks.sort_values(ascending=False)

id_student
80868     28615
630905    28264
537811    25159
619927    24368
434476    21204
          ...  
623319        1
558146        1
682505        1
146089        1
576481        1
Name: sum_click, Length: 26074, dtype: int64

#### Student 28400

In [61]:
id_28400 = student_vle[student_vle['id_student']=='28400']

In [63]:
len(id_28400.id_site.unique())

84

In [66]:
id_28400.code_module.unique()

array(['AAA'], dtype=object)

In [67]:
id_28400.sum_click.sum()

1435